# Аугментация данных

В нашем примере данные были сбалансированными. А как работать с несбалансированными данными?

Рассмотрим задачу распознавания тональности твитов, взятых из [Twitter Sentimental Analysis challenge](https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/).

Источник изложения: https://github.com/mabusalah/Resampling

Получим данные

In [1]:
!wget https://www.dropbox.com/s/3eefrxvfin11m5p/test_tweets.csv
!wget https://www.dropbox.com/s/rzpc5gxlvl6zh8a/train.csv

--2023-09-26 20:54:51--  https://www.dropbox.com/s/3eefrxvfin11m5p/test_tweets.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/3eefrxvfin11m5p/test_tweets.csv [following]
--2023-09-26 20:54:51--  https://www.dropbox.com/s/raw/3eefrxvfin11m5p/test_tweets.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc36829f8fc8b6584b088e4faec7.dl.dropboxusercontent.com/cd/0/inline/CEcYktsT5e3Z40BNDhFLX04CCW6VLg8XqK3aZPAV1jEF6BOKMEujI_5Coek6UephROT83gC6XWNk_zkGphlL-u_pKUUPmqRwQQ4drwO4e8aNi213ttOAvMFKmdac0jMqZh4/file# [following]
--2023-09-26 20:54:52--  https://uc36829f8fc8b6584b088e4faec7.dl.dropboxusercontent.com/cd/0/inline/CEcYktsT5e3Z40BNDhFLX04CCW6VLg8XqK3aZPAV1jEF6BOKMEujI_5Coek6UephROT83gC6XWNk_zkGphlL-u_pKUUPmqRwQQ4drwO4e8aNi213tt

In [2]:
import pandas as pd
test = pd.read_csv('test_tweets.csv')
print("Test Set:"% test.columns, test.shape, len(test))
train = pd.read_csv('train.csv')
print("Training Set:"% train.columns, train.shape, len(train))

Test Set: (17197, 2) 17197
Training Set: (31962, 3) 31962


In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


Итак, посмотрим, какой процент от общей выборки занимают позитивные и негативные примеры.

In [5]:
print("Positive: ", train.label.value_counts()[0]/len(train)*100,"%")
print("Negative: ", train.label.value_counts()[1]/len(train)*100,"%")

Positive:  92.98542018647143 %
Negative:  7.014579813528565 %


93% vs. 7% - данные определенно несбалансированны, что, в свою очередь, негативно влияет на точность предсказания.
Для начала поработаем с исходными данными и оценим точность классификации.
Начнем с предобработки данных: уберем из твитов числа, html/xml-тэги, специальные символы.

In [6]:
import re
from bs4 import BeautifulSoup #для работы с html/xml-тэгами
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer

porter=PorterStemmer()
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()

    words = tok.tokenize(lower_case)

    stem_sentence=[]
    for word in words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    words="".join(stem_sentence).strip()
    return words

nums = [0,len(train)]
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    clean_tweet_texts.append(tweet_cleaner(train['tweet'][i]))

nums = [0,len(test)]
test_tweet_texts = []

for i in range(nums[0],nums[1]):
    test_tweet_texts.append(tweet_cleaner(test['tweet'][i]))

train_clean = pd.DataFrame(clean_tweet_texts,columns=['tweet'])
train_clean['label'] = train.label
train_clean['id'] = train.id
test_clean = pd.DataFrame(test_tweet_texts,columns=['tweet'])
test_clean['id'] = test.id

<ipython-input-6-2a8a31033b1e>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


Разделим данные на обучающие и проверочные.

In [7]:
from sklearn import model_selection, preprocessing, metrics, linear_model, svm

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_clean['tweet'],train_clean['label'])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

Рассчитаем TF-IDF признаки.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(train_clean['tweet'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

Попробуйте использовать обычный счетчик слов для извлечения признаков.

Точность в качестве метрики работает хорошо только на сбалансированных наборах данных, поэтому для оценки результатов работы  алгоритма будем использовать F1-метрику.

In [9]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)

    predictions = classifier.predict(feature_vector_valid)

    return metrics.f1_score(valid_y,predictions)

Для начала обучим лог-регрессию.

In [10]:
accuracyORIGINAL = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),xtrain_tfidf, train_y, xvalid_tfidf)
print ("Logistic regression Baseline, WordLevel TFIDF: ", accuracyORIGINAL)

Logistic regression Baseline, WordLevel TFIDF:  0.5375796178343949


Как видно, результат оставляет желать лучшего.

Что можно сделать с данными?

Было бы неплохо как-то увеличить  количество негативных примеров, или же уменьшить количество положительных. Для этого существуют различные техники аугментации данных.
В Python для этих целей есть библиотека imblearn (imbalanced-learn).

In [11]:
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler,
                                    NearMiss,
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline

Итак, в качестве инструментов для аугментации рассмотрим: under-sampling, over-sampling и их комбинацию.

**Under-sampling** уравновешивает данные за счет уменьшения размера  превалирующего класса.
Этот метод разумно использовать, когда количество данных достаточно велико, иначе есть риск остаться и вовсе без обучающих примеров.

Итак, логика действия довольно проста: мы просто случайным образом убираем лишние экземпляры из превалирующего класса.

Так как в нашем примере лишь 7% всех твитов имеют негативную окраску, уравновешивание позитивного набора с этими 7-ю процентами вряд ли обеспечит хороший результат.

Попробуем...

In [12]:
rus = RandomUnderSampler(random_state=0, replacement=True)
rus_xtrain_tfidf, rus_train_y = rus.fit_resample(xtrain_tfidf, train_y)
accuracyrus = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),rus_xtrain_tfidf, rus_train_y, xvalid_tfidf)
print ("Logistic regression RUS, WordLevel TFIDF: ", accuracyrus)

Logistic regression RUS, WordLevel TFIDF:  0.4969325153374233


Действительно, все стало только хуже.

Попробуем другие алгоритмы **under-sampling**.

Например, **NearMiss**. Данный алгоритм выбирает, какие экземпляры нужно оставить в превалирующем классе на основании некоторых эвристик. Существует три варианта данного алгоритма:

**NearMiss-1** оставляет те экземпляры из превалирующего класса, для которых среднее расстояние до *k* ближайших соседей из миноритарного класса будет наименьшим.

**NearMiss-2** оставляет те экземпляры из превалирующего класса, для которых среднее расстояние до *k* самых дальних соседей из миноритарного класса будет наименьшим.

**NearMiss-3** состоит из двух шагов: сначала, для каждого экземпляра из миноритарного класса выбирается *k* ближайших соседей из превалирующего класса, затем из большего класса выбираются те экземпляры, для которых среднее расстояние до *k* ближайших соседей максимальное.

In [13]:
for sampler in (NearMiss(version=1),NearMiss(version=2),NearMiss(version=3)):
    nm_xtrain_tfidf, nm_train_y = sampler.fit_resample(xtrain_tfidf, train_y)
    accuracysm = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),nm_xtrain_tfidf, nm_train_y, xvalid_tfidf)
    print ("Logistic regression NearMiss(version= {0}), WordLevel TFIDF: ".format(sampler.version), accuracysm)

Logistic regression NearMiss(version= 1), WordLevel TFIDF:  0.24621494167287167
Logistic regression NearMiss(version= 2), WordLevel TFIDF:  0.5147991543340381
Logistic regression NearMiss(version= 3), WordLevel TFIDF:  0.5373687461395924


**Edited Nearest Neighbor (ENN)**

ENN удаляет из большего класса элемент, если класс его ближайшего соседа отличается от его собственного.

In [ ]:
enn_xtrain_tfidf, enn_train_y = EditedNearestNeighbours().fit_resample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),enn_xtrain_tfidf, enn_train_y, xvalid_tfidf)
print ("Logistic regression {0}, WordLevel TFIDF: ", accuracy)

Logistic regression {0}, WordLevel TFIDF:  0.542713567839196


Как вы поняли, при применении **Under-sampling** техник новые данные не генерируются, в отличие от **Over-sampling**.

# Over-sampling

Итак, когда данных недостаточно или количество экземпляров в миноритарном классе очень мало, применяется **Over-sampling**.

При применении этой техники балансировка данных происходит за счет увеличения количества экземпляров в миноритарном классе. Новые элементы генерируются за счет: повторения, бутстрэппинга, SMOTE (Synthetic Minority Over-Sampling Technique) или ADASYN (Adaptive synthetic sampling).

**Random Over-sampling**: случайным образом дублируются некоторые элементы из миноритарного класса.

In [ ]:
#Random Over Sampling
ros = RandomOverSampler(random_state=777)
ros_xtrain_tfidf, ros_train_y = ros.fit_resample(xtrain_tfidf, train_y)
accuracyROS = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("Logistic regression ROS, WordLevel TFIDF: ", accuracyROS)

Logistic regression ROS, WordLevel TFIDF:  0.6627712854757929


**SMOTE Over-sampling**

Алгоритм SMOTE основан на идее генерации некоторого количества искусственных примеров, которые были бы «похожи» на имеющиеся в миноритарном классе, но при этом не дублировали их.

Для создания новой записи находят разность $d=X_b-X_a,$ где $ X_b, X_a -$ векторы признаков «соседних» примеров $a$ и $b$ из миноритарного класса.

Их находят, используя алгоритм ближайшего соседа (*KNN*). В данном случае необходимо и достаточно для примера $b$ получить набор из $k$ соседей, из которого в дальнейшем будет выбрана запись $b$. Остальные шаги алгоритма *KNN* не требуются.

Далее из $d$ путем умножения каждого его элемента на случайное число в интервале (0, 1) получают $\hat{d}$. Вектор признаков нового примера вычисляется путем сложения $X_a$ и $\hat{d}$.

Алгоритм **SMOTE** позволяет задавать количество записей, которое необходимо искусственно сгенерировать. Степень сходства примеров $a$ и $b$ можно регулировать путем изменения значения $k$ (числа ближайших соседей).

In [ ]:
sm = SMOTE(random_state=777)
sm_xtrain_tfidf, sm_train_y = sm.fit_resample(xtrain_tfidf, train_y)
accuracySMOTE = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),sm_xtrain_tfidf, sm_train_y, xvalid_tfidf)
print ("Logistic regression SMOTE, WordLevel TFIDF: ", accuracySMOTE)

Logistic regression SMOTE, WordLevel TFIDF:  0.6661002548853017


Итак, по сравнению с **Random Over-sampling** разница небольшая.

Проверьте результаты **Random Over-sampling** и **SMOTE Over-sampling** для реальных тестовых данных (*test_clean*).

Следующий алгоритм **ASMO: Adaptive synthetic minority oversampling**.



Сгенерировать искусственные записи в пределах отдельных кластеров на основе всех классов. Для каждого примера миноритарного класса находят m ближайших соседей, и на основе них (также как в SMOTE) создаются новые записи.

1.   Если для каждого $i$-ого примера миноритарного класса из $k$ ближайших соседей $g$ ($g\leq k$) принадлежит к мажоритарному, то набор данных считается «рассеянным». В этом случае используют алгоритм **ASMO**, иначе применяют **SMOTE** (как правило, $g$ задают равным 20).
2.   Используя только примеры миноритарного класса, выделить несколько кластеров (например, алгоритмом $k$-means).
3.   Сгенерировать искусственные записи в пределах отдельных кластеров на основе всех классов. Для каждого примера миноритарного класса находят m ближайших соседей, и на основе них (также как в **SMOTE**) создаются новые записи.

Такая модификация алгоритма **SMOTE** делает его более адаптивным к различным наборам данных с несбалансированными классами.

In [ ]:
ad = ADASYN(random_state=777)
ad_xtrain_tfidf, ad_train_y = ad.fit_resample(xtrain_tfidf, train_y)
accuracyADASYN = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ad_xtrain_tfidf, ad_train_y, xvalid_tfidf)
print ("Logistic regression ADASYN, WordLevel TFIDF: ", accuracyADASYN)

Logistic regression ADASYN, WordLevel TFIDF:  0.6524239934264585


И опять проверим на реальных тестовых примерах.

# Комбинация **Under-** и **Over-sampling**

В *imblearn* реализованы две возможные комбинации:


1.   **SMOTE** + **ENN**
2.   **SMOTE** + **Tomek Link Removal** (Пара двух ближайших соседей, которые принадлежат разным классам называется *Tomek link*. Under-sampling заключается в удалении всех таких элементов из мажоритарного класса)

Подробнее: https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.combine



In [ ]:
se = SMOTEENN(random_state=42)
se_xtrain_tfidf, se_train_y = se.fit_resample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),se_xtrain_tfidf, se_train_y, xvalid_tfidf)
print ("Logistic regression SMOTEENN: ", accuracy)

Logistic regression SMOTEENN:  0.37485626676887696


Первый метод сработал плохо. Оцените работу второго подхода.

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression SMOTEENN:  0.6945337620578779


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
